In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a href="notebook_id"></a>
# Implementing a simple generic class

If you review the implementation of `FixedStringArrayList` you may notice one striking thing about the methods: None of the methods (except `toString` if the reader has attempted the exercises) use any `String` methods. In fact, if you wanted to create a class that represents a fixed-size list of `Integer`s then all you would have to do is replace every occurrence of `String` with `Integer` (except for a few places in the `main` method).

### Exercise

1. Implement a `FixedIntegerArrayList` class by copying the `FixedStringArrayList` implementation and changing occurrences of `String` to `Integer`.

The fact that the list implementation does not depend on the element type of the list is a strong hint that the class could be made generic with little effort.

As a reminder, a generic class is a class that has one or more type parameters. For example, the Java library class `ArrayList` is a list that is parameterized by its element type:

In [ ]:
import java.util.ArrayList;

ArrayList<Integer>   t1 = new ArrayList<>();   // list of Integer
ArrayList<String>    t2 = new ArrayList<>();   // list of String
ArrayList<Character> t3 = new ArrayList<>();   // list of Character

Instead of creating a separate class for every element type (which would involve a lot of copying), the `ArrayList` class is a single class where the element type becomes a parameter of the implementation.

To create a generic class the implementer chooses names for the type parameters. For classes that implement collections of elements the name `E` is traditional. The type parameter is written inside the delimiters `<>` after the class name.

We will now convert the `FixedStringArrayList` class to a generic class `FixedArrayList`. We begin with the class declaration:

In [ ]:
package ca.queensu.cs.cisc235.list;

import java.util.Arrays;

/**
 * A minimal implementation of a fixed-size list of elements backed by an array.
 *
 * @param <E> the element type of this list
 */
public class FixedArrayList<E> {
    
    
}

Notice the type parameter `<E>` after the class name.

The type of the array field now must change from `String[]` to `E[]` because now we have an array of elements of type `E`. The field `back` does not change.

In [ ]:
package ca.queensu.cs.cisc235.list;

import java.util.Arrays;

/**
 * A minimal implementation of a fixed-size list of elements backed by an array.
 *
 * @param <E> the element type of this list
 */
public class FixedArrayList<E> {
    
    /**
     * The array of strings in the list.
     */
    private E[] arr;
    
    /**
     * Index of the last element in the list; equal to -1 for an empty list.
     */
    private int back;     
}

The constructor must change because we no longer have an array of strings. One of the unusual things about generic types is that the compiler will not allow the programmer to create an array of generic type; the reason for this is because generics were added to the Java language some time after the language became widely adopted. Interested readers are encouraged to read Item 28 of *Effective Java, 3rd Edition* by Joshua Bloch.

The solution is to create an array of `Object` and then cast the array to the generic type `E[]`. An array of `Object` works to hold elements of any reference type because `Object` is the superclass of all other classes in Java.

In [ ]:
package ca.queensu.cs.cisc235.list;

import java.util.Arrays;

/**
 * A minimal implementation of a fixed-size list of elements backed by an array.
 *
 * @param <E> the element type of this list
 */
public class FixedArrayList<E> {

    /**
     * The array of strings in the list.
     */
    private E[] arr;
    
    /**
     * Index of the last element in the list; equal to -1 for an empty list.
     */
    private int back;      
    
    /**
     * Initializes this list to an empty list of capacity 16.
     */
    public FixedArrayList() {
        this.arr = (E[]) (new Object[16]);     // make an array of Object and then cast to E[]
        this.back = -1;
    }
    
}

And now we are more or less done except that we need to change some `String` variables to variables of type `E`. The complete implementation is shown in the following cell. Run the cell to compile the class and then run the following cell to run the `main` method.

In [ ]:
package ca.queensu.cs.cisc235.list;

import java.util.Arrays;

/**
 * A minimal implementation of a fixed-size list of elements backed by an array.
 *
 * @param <E> the element type of this list
 */
public class FixedArrayList<E> {

    /**
     * The array of strings in the list.
     */
    private E[] arr;
    
    /**
     * Index of the last element in the list; equal to -1 for an empty list.
     */
    private int back;      
    
    /**
     * Initializes this list to an empty list of capacity 16.
     */
    public FixedArrayList() {
        this.arr = (E[]) (new Object[16]);
        this.back = -1;
    }
    
    /**
     * Returns the number of elements in this list.
     * 
     * @return the number of elements in this list
     */
    public int size() {
        return this.back + 1;
    }
    
    /**
     * Test if an index is valid for this list.
     * @param index
     */
    private void checkIndex(int index) {
        if (index < 0 || index >= this.size()) {
            throw new IndexOutOfBoundsException(index + " out of bounds");
        }
    }
    
    /**
     * Returns the element at the specified index of this list.
     * 
     * @param index the index of the element to get
     * @return the element at the specified index of this list
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E get(int index) {
        this.checkIndex(index);
        return this.arr[index];
    }
    
    /**
     * Sets the element at the specified index to the specified element in this
     * list returning the old element.
     *   
     * @param index the index of the element to set
     * @param elem the new element
     * @return the old element
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E set(int index, E elem) {
        this.checkIndex(index);
        E old = this.arr[index];
        this.arr[index] = elem;
        return old;
    }
    
    /**
     * Adds the specified element to the end of this list.
     * 
     * @param elem the element to add
     * @throws RuntimeException if the list is full
     */
    public void add(E elem) {
        if (this.size() == this.arr.length) {
            throw new RuntimeException("list is full");
        }
        // adjust back
        this.back++;
        this.arr[this.back] = elem;
    }
    
    /**
     * Removes the element at the specified index of this list returning 
     * the element that was removed. All elements after the specified index
     * are shifted one position to the front of the list.
     * 
     * @param index the index of the element to remove
     * @return the removed element
     * @throws IndexOutOfBoundsException if index is out of bounds
     */
    public E remove(int index) {
        this.checkIndex(index);
        E old = this.arr[index];
        
        // shift elements from (index + 1) down one index
        for (int i = index + 1; i <= this.back; i++) {
            this.arr[i - 1] = this.arr[i]; 
        }
        
        // null out old back element, otherwise it can't be garbage collected
        this.arr[this.back] = null;
        
        // adjust back
        this.back--;
        
        return old;
    }
    
    
    
    public static void main(String[] args) {
        FixedArrayList<String> t = new FixedArrayList<>();
        for (char c = 'a'; c < 'a' + 16; c++) {
            t.add("" + c);
        }
        System.out.println(Arrays.toString(t.arr));
        
        for (int i = 0; i < 16; i++) {
            System.out.println(t.get(i));
        }
        
        for (int i = 0; i < 16; i++) {
            String s = t.get(i);
            t.set(i, s + s);
        }
        System.out.println(Arrays.toString(t.arr));
        
        for (int i = 0; i < 16; i++) {
            t.remove(0);
            System.out.println(Arrays.toString(t.arr));
        }
        
        // force an exception
        t.set(0, "");
    }
}


In [ ]:
ca.queensu.cs.cisc235.list.FixedArrayList.main(null);

## Exercises

1. Modify the `main` method to create and manipulate `FixedArrayList`s of different element types. Does the `main` method produce the expected output?

2. Implement `equals(Object)`, `hashCode`, and `toString` for the `FixedArrayList` class. Two lists are equal if they have equal sizes and contain equal strings in the same order. For `toString` reproduce the way that Java lists implement `toString`. See the following note regarding the implementation of `equals`.

### Note

When implementing `equals` you will want to write something like the following:

```java
	@Override
	public boolean equals(Object obj) {
		if (this == obj) {
			return true;
		}
		if (!(obj instanceof FixedArrayList<E>)) {
			return false;
		}
		FixedArrayList<E> other = (FixedArrayList<E>) obj;
     
        // compare this.arr and other.arr here
    }
```

If you try to do this the compiler will emit a compilation error on the line:

```java
		if (!(obj instanceof FixedArrayList<E>)) {
```

The compiler error is caused because of the way that Java implements generics. At runtime, the generic type parameter is erased so there is no way to check if `obj` is an instance of a list of strings or a list of integers. The best that you can do is ask if `obj` is a `FixedArrayList` of something where you don't care what the exact type is. You do this using a question mark:

```java
		if (!(obj instanceof FixedArrayList<?>)) {
```

where the `?` is a wild-card type that means "any type".

Similarly, because of generic type erasure the cast is incorrect as well. Again, the best that you can do in this situation is to use `?`:

```java
	@Override
	public boolean equals(Object obj) {
		if (this == obj) {
			return true;
		}
		if (!(obj instanceof FixedArrayList<?>)) {
			return false;
		}
		FixedArrayList<?> other = (FixedArrayList<?>) obj;
     
        // compare this.arr and other.arr here
    }
```